In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
import matplotlib
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.sans-serif'] = ['FreeSans']
import seaborn as sns
import itertools
from tqdm import tqdm
import joblib

In [2]:
d0 = np.array([0.5, 0.5])

In [3]:
Rs = [
    np.array([[1., 2.], [0., 0.]]),
    np.array([[-1., 1.], [0., 0.]]),
    np.array([[-1., -2.], [0., 0.]]),
]
sigmas = [
    np.array([[0.5, 0.5], [0.5, 0.5]]),
    np.array([[0.5, 0.5], [0.5, 0.5]]),
    np.array([[0.5, 0.5], [0.5, 0.5]]),
]

In [4]:
πs = [
    np.array([[1., 0.], [1., 0.]]),
    np.array([[0., 1.], [1., 0.]]),
    np.array([[0.5, 0.5], [1., 0.]]),
    np.array([[0.1, 0.9], [1., 0.]]),
    np.array([[0.8, 0.2], [1., 0.]]),
]

In [5]:
use_πD = False

In [6]:
N = 1
runs = 1000

In [7]:
def single_exp_setting(π_b, π_e):
    np.random.seed(42)

    # True value of π_e
    Js = []
    for seed in range(runs):
        rng = np.random.default_rng(seed=10+seed)
        x = rng.choice(len(d0), size=N, p=d0)
        a = np.array([rng.choice(2, p=π_e[xi]) for xi in x])
        r = np.array([rng.normal(R[xi,ai], sigma[xi,ai]) for xi,ai in zip(x,a)])
        J = np.sum(r) / N
        Js.append(J)

    # Standard IS
    Gs = []
    OISs = []
    WISs = []
    for seed in range(runs):
        rng = np.random.default_rng(seed=10+seed)
        x = rng.choice(len(d0), size=N, p=d0)
        a = np.array([rng.choice(2, p=π_b[xi]) for xi in x])
        r = np.array([rng.normal(R[xi,ai], sigma[xi,ai]) for xi,ai in zip(x,a)])
        G = np.sum(r) / N
        Gs.append(G)

        if use_πD:
            assert False
        else:
            π_b_ = π_b

        rho = π_e[x,a] / π_b_[x,a]
        OISs.append(np.sum(rho * r) / N)
        WISs.append(np.sum(rho * r) / np.sum(rho))

    
    # Collect data using π_b - naive approach
    Naive_OISs = []
    for seed in range(runs):
        rng = np.random.default_rng(seed=10+seed)
        rng_c = np.random.default_rng(seed=100000+seed)
        x = rng.choice(len(d0), size=N, p=d0)
        a = np.array([rng.choice(2, p=π_b[xi]) for xi in x])
        r = np.array([rng.normal(R[xi,ai], sigma[xi,ai]) for xi,ai in zip(x,a)])
        rho = π_e[x,a] / π_b[x,a]

        # counterfactual flag
        c = np.array([rng_c.choice(2, p=[1-Pc[xi,ai], Pc[xi,ai]]) for xi,ai in zip(x,a)])

        # counterfactual reward
        rc = np.array([rng_c.normal(R[xi,1-ai], sigma[xi,1-ai]) for xi,ai in zip(x,a)])
        rc[c==0] = np.nan

        # trajectory-wise weight
        w = np.ones(N)
        w[c==1] = ww_naive[x[c==1], a[c==1], a[c==1]]
        wc = np.zeros(N)
        wc[c==1] = ww_naive[x[c==1], a[c==1], 1-a[c==1]]

        if use_πD:
            # augmented behavior policy
            assert False
        else:
            # augmented behavior policy
            π_b_ = np.array([
                [π_b[0,0]*ww_naive[0,0,0]+π_b[0,1]*ww_naive[0,1,0], π_b[0,0]*ww_naive[0,0,1]+π_b[0,1]*ww_naive[0,1,1]],
                [π_b[1,0]*ww_naive[1,0,0]+π_b[1,1]*ww_naive[1,1,0], π_b[1,0]*ww_naive[1,0,1]+π_b[1,1]*ww_naive[1,1,1]],
            ])
            π_b_ = π_b_ / π_b_.sum(axis=1, keepdims=True)

        # Naive_WISs.append(
        #     (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (np.sum(w* π_e[x,a] / π_b_[x,a]) + np.sum((wc* π_e[x,1-a] / π_b_[x,1-a])[c==1]))
        # )
        ## Add factual and counterfactual separately
        Naive_OISs.append(np.sum(π_e[x,a] / π_b_[x,a] * r) / N)
        if np.sum(c) > 0:
            Naive_OISs.append(np.sum(π_e[x,1-a] / π_b_[x,1-a] * rc) / np.sum(c))


    # Collect data using π_b - combining counterfactuals with factuals
    FC_OISs_w = []
    FC_WISs_w = []
    for seed in range(runs):
        rng = np.random.default_rng(seed=10+seed)
        rng_c = np.random.default_rng(seed=100000+seed)
        x = rng.choice(len(d0), size=N, p=d0)
        a = np.array([rng.choice(2, p=π_b[xi]) for xi in x])
        r = np.array([rng.normal(R[xi,ai], sigma[xi,ai]) for xi,ai in zip(x,a)])
        rho = π_e[x,a] / π_b[x,a]

        # counterfactual flag
        c = np.array([rng_c.choice(2, p=[1-Pc[xi,ai], Pc[xi,ai]]) for xi,ai in zip(x,a)])

        # counterfactual reward
        rc = np.array([rng_c.normal(R[xi,1-ai], sigma[xi,1-ai]) for xi,ai in zip(x,a)])
        rc[c==0] = np.nan

        # trajectory-wise weight
        w = np.ones(N)
        w[c==1] = ww[x[c==1], a[c==1], a[c==1]]
        wc = np.zeros(N)
        wc[c==1] = ww[x[c==1], a[c==1], 1-a[c==1]]

        if use_πD:
            # augmented behavior policy
            assert False
        else:
            # augmented behavior policy
            π_b_ = np.array([
                [π_b[0,0]*ww[0,0,0]+π_b[0,1]*ww[0,1,0], π_b[0,0]*ww[0,0,1]+π_b[0,1]*ww[0,1,1]],
                [π_b[1,0]*ww[1,0,0]+π_b[1,1]*ww[1,1,0], π_b[1,0]*ww[1,0,1]+π_b[1,1]*ww[1,1,1]],
            ])
            π_b_ = π_b_ / π_b_.sum(axis=1, keepdims=True)

        FC_OISs_w.append(
            (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (N)
        )
        FC_WISs_w.append(
            (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (np.sum(w* π_e[x,a] / π_b_[x,a]) + np.sum((wc* π_e[x,1-a] / π_b_[x,1-a])[c==1])),
        )

    df_bias_var = []
    for name, values in [
        ('$\hat{v}(\pi_e)$', Js),
        ('$\hat{v}(\pi_b)$', Gs),
        ('OIS', OISs),
        ('WIS', WISs),
        ('C-OIS', FC_OISs_w),
        ('C-WIS', FC_WISs_w),
        ('Naive-OIS', Naive_OISs),
    ]:
        df_bias_var.append([name, 
                            np.mean(values), 
                            np.mean(values - d0@np.sum(π_e*R, axis=1)), 
                            np.sqrt(np.var(values)), 
                            np.sqrt(np.mean(np.square(values - d0@np.sum(π_e*R, axis=1))))])
    return pd.DataFrame(df_bias_var, columns=['Approach', 'Mean', 'Bias', 'Std', 'RMSE'])

# Ideal counterfactual annotations, equal weights

In [8]:
# Counterfactual-augmented IS
## probability of getting a counterfactual annotation
Pc = np.array([
    [1., 1.],
    [0., 0.],
])
## Weights assigned to factual and counterfactuals
ww = np.array([
    [[0.5, 0.5], [0.5, 0.5]],
    [[1, 0], [0, 1]],
])

ww_naive = np.array([
    [[1, 1], [1, 1]],
    [[1, 0], [0, 1]],
])

## Rs[0] setting

In [9]:
R, sigma = Rs[0], sigmas[0]

In [10]:
df_out_all_0 = []
for π_b, π_e in tqdm(list(itertools.product(πs, πs))):
    df_out = single_exp_setting(π_b, π_e)
    df_out_all_0.append(df_out)

  0%|          | 0/25 [00:00<?, ?it/s]<ipython-input-7-93125942f8f4>:115: RuntimeWarning: invalid value encountered in true_divide
  (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (N)
<ipython-input-7-93125942f8f4>:118: RuntimeWarning: invalid value encountered in true_divide
  (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (np.sum(w* π_e[x,a] / π_b_[x,a]) + np.sum((wc* π_e[x,1-a] / π_b_[x,1-a])[c==1])),
  4%|▍         | 1/25 [00:07<02:51,  7.13s/it]<ipython-input-7-93125942f8f4>:33: RuntimeWarning: invalid value encountered in double_scalars
  WISs.append(np.sum(rho * r) / np.sum(rho))
100%|██████████| 25/25 [02:02<00:00,  4.89s/it]


In [11]:
df_tmp = pd.DataFrame(index=[str(π)[1:-1] for π in πs], columns=[str(π)[1:-1] for π in πs])
df_tmp.index.name = 'π_b'
df_tmp.columns.name = 'π_e'
for (i, π_b), (j, π_e) in itertools.product(enumerate(πs), enumerate(πs)):
    ix = i*len(πs)+j
    df_tmp.iloc[i,j] = str(df_out_all_0[ix].iloc[[2,6,4], [2,3,4]].round(1).values)[1:-1]
display(df_tmp.style.set_properties(**{'white-space': 'pre-wrap'}))

π_e,[1. 0.] [1. 0.],[0. 1.] [1. 0.],[0.5 0.5] [1. 0. ],[0.1 0.9] [1. 0. ],[0.8 0.2] [1. 0. ]
π_b,,,,,
[1. 0.] [1. 0.],[0. 0.7 0.7] [0.2 1.1 1.2] [0. 0.7 0.7],[-1. 0.4 1. ] [ 0.3 2. 2. ] [ 0. 1.1 1.1],[-0.5 0.5 0.7] [ 0.3 0.9 1. ] [ 0. 0.9 0.9],[-0.9 0.4 1. ] [ 0.3 1.8 1.8] [ 0. 1.1 1.1],[-0.2 0.6 0.6] [ 0.2 0.9 0.9] [ 0. 0.7 0.7]
[0. 1.] [1. 0.],[-0.5 0.4 0.6] [ 0.2 1.1 1.1] [ 0. 0.7 0.7],[0. 1.1 1.1] [0.3 2. 2. ] [0. 1.1 1.1],[-0.2 0.6 0.7] [ 0.3 0.9 1. ] [ 0. 0.9 0.9],[-0. 1. 1. ] [ 0.3 1.8 1.8] [ 0. 1.1 1.1],[-0.4 0.4 0.6] [ 0.2 0.8 0.9] [ 0. 0.7 0.7]
[0.5 0.5] [1. 0. ],[0. 1. 1. ] [0.2 1.1 1.1] [0. 0.7 0.7],[-0. 1.8 1.8] [ 0.3 2. 2. ] [ 0. 1.1 1.1],[0. 1. 1. ] [0.3 0.9 1. ] [0. 0.9 0.9],[-0. 1.6 1.6] [ 0.3 1.8 1.8] [ 0. 1.1 1.1],[0. 0.8 0.8] [0.2 0.8 0.9] [0. 0.7 0.7]
[0.1 0.9] [1. 0. ],[0.1 2.6 2.6] [0.2 1.1 1.1] [0. 0.7 0.7],[-0. 1.2 1.2] [ 0.3 2. 2. ] [ 0. 1.1 1.1],[0. 1.3 1.3] [0.3 0.9 1. ] [0. 0.9 0.9],[0. 1.1 1.1] [0.3 1.8 1.8] [0. 1.1 1.1],[0.1 2. 2. ] [0.2 0.8 0.9] [0. 0.7 0.7]
[0.8 0.2] [1. 0. ],[0. 0.8 0.8] [0.2 1.1 1.2] [0. 0.7 0.7],[0.1 3.2 3.2] [0.3 2. 2. ] [0. 1.1 1.1],[0. 1.6 1.6] [0.3 0.9 1. ] [0. 0.9 0.9],[0. 2.9 2.9] [0.3 1.7 1.8] [0. 1.1 1.1],[0. 0.8 0.8] [0.2 0.8 0.9] [0. 0.7 0.7]


In [ ]:
for (i, π_b) in enumerate(πs):
    print("""$[\mask{{0.0}}{{{}}},\mask{{0.0}}{{{}}}]$ """.format(π_b[0,0], π_b[0,1]))
    for (j, π_e) in enumerate(πs):
        ix = i*len(πs)+j
        values = df_out_all_0[ix].iloc[[2,6,4], [2,3,4]].round(2).values
        print("""& \\scalebox{0.8}{$\\begin{matrix} """
              + """\mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}}""".format(*list(values.ravel())).replace('-', '\\shortminus ')
              + """ \end{matrix}$} """)
    print("""\\\\[12pt]""")

In [13]:
for (i, π_b) in enumerate(πs):
    print("""$[\mask{{0.0}}{{{}}},\mask{{0.0}}{{{}}}]$ """.format(π_b[0,0], π_b[0,1]))
    for (j, π_e) in enumerate(πs):
        ix = i*len(πs)+j
        values = df_out_all_0[ix].iloc[[2,6,4], [2,3,4]].round(1).values
        print("""& \\scalebox{0.8}{$\\begin{matrix} """
              + """\mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}}""".format(
                  *[int(x) if x.is_integer() else x for x in list(values.ravel())]).replace('-', '\\shortminus ')
              + """ \end{matrix}$} """)
    print("""\\\\[12pt]""")

$[\mask{0.0}{1.0},\mask{0.0}{0.0}]$ 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{0} & \mask{0.00}{0.7} & \mask{0.00}{0.7} \\ \mask{0.00}{0.2} & \mask{0.00}{1.1} & \mask{0.00}{1.2} \\ \mask{0.00}{0} & \mask{0.00}{0.7} & \mask{0.00}{0.7} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{\shortminus 1} & \mask{0.00}{0.4} & \mask{0.00}{1} \\ \mask{0.00}{0.3} & \mask{0.00}{2} & \mask{0.00}{2} \\ \mask{0.00}{0} & \mask{0.00}{1.1} & \mask{0.00}{1.1} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{\shortminus 0.5} & \mask{0.00}{0.5} & \mask{0.00}{0.7} \\ \mask{0.00}{0.3} & \mask{0.00}{0.9} & \mask{0.00}{1} \\ \mask{0.00}{0} & \mask{0.00}{0.9} & \mask{0.00}{0.9} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{\shortminus 0.9} & \mask{0.00}{0.4} & \mask{0.00}{1} \\ \mask{0.00}{0.3} & \mask{0.00}{1.8} & \mask{0.00}{1.8} \\ \mask{0.00}{0} & \mask{0.00}{1.1} & \mask{0.00}{1.1} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{\shortminus 0.2} & \mask

## Rs[1] setting

In [14]:
R, sigma = Rs[1], sigmas[1]

In [15]:
df_out_all_1 = []
for π_b, π_e in tqdm(list(itertools.product(πs, πs))):
    df_out = single_exp_setting(π_b, π_e)
    df_out_all_1.append(df_out)

  0%|          | 0/25 [00:00<?, ?it/s]<ipython-input-7-93125942f8f4>:115: RuntimeWarning: invalid value encountered in true_divide
  (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (N)
<ipython-input-7-93125942f8f4>:118: RuntimeWarning: invalid value encountered in true_divide
  (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (np.sum(w* π_e[x,a] / π_b_[x,a]) + np.sum((wc* π_e[x,1-a] / π_b_[x,1-a])[c==1])),
  4%|▍         | 1/25 [00:04<01:41,  4.24s/it]<ipython-input-7-93125942f8f4>:33: RuntimeWarning: invalid value encountered in double_scalars
  WISs.append(np.sum(rho * r) / np.sum(rho))
100%|██████████| 25/25 [02:03<00:00,  4.93s/it]


In [16]:
df_tmp = pd.DataFrame(index=[str(π)[1:-1] for π in πs], columns=[str(π)[1:-1] for π in πs])
df_tmp.index.name = 'π_b'
df_tmp.columns.name = 'π_e'
for (i, π_b), (j, π_e) in itertools.product(enumerate(πs), enumerate(πs)):
    ix = i*len(πs)+j
    df_tmp.iloc[i,j] = str(df_out_all_1[ix].iloc[[2,6,4], [2,3,4]].round(1).values)[1:-1]
display(df_tmp.style.set_properties(**{'white-space': 'pre-wrap'}))

π_e,[1. 0.] [1. 0.],[0. 1.] [1. 0.],[0.5 0.5] [1. 0. ],[0.1 0.9] [1. 0. ],[0.8 0.2] [1. 0. ]
π_b,,,,,
[1. 0.] [1. 0.],[ 0. 0.7 0.7] [-0.1 1.1 1.1] [ 0. 0.7 0.7],[-0.5 0.4 0.6] [ 0.2 1.1 1.1] [ 0. 0.7 0.7],[-0.2 0.5 0.5] [ 0. 0.9 0.9] [ 0. 0.4 0.4],[-0.4 0.4 0.6] [ 0.2 1.1 1.1] [ 0. 0.6 0.6],[-0.1 0.6 0.6] [-0.1 1. 1. ] [ 0. 0.6 0.6]
[0. 1.] [1. 0.],[ 0.5 0.4 0.6] [-0.1 1.1 1.1] [ 0. 0.7 0.7],[0. 0.7 0.7] [0.2 1.1 1.2] [0. 0.7 0.7],[0.3 0.5 0.5] [0. 0.9 0.9] [0. 0.4 0.4],[0.1 0.6 0.7] [0.2 1.1 1.1] [0. 0.6 0.6],[ 0.4 0.4 0.6] [-0.1 1. 1. ] [ 0. 0.6 0.6]
[0.5 0.5] [1. 0. ],[ 0. 1.1 1.1] [-0.1 1.1 1.1] [ 0. 0.7 0.7],[0. 1.1 1.1] [0.2 1.2 1.2] [0. 0.7 0.7],[0. 0.9 0.9] [0. 1. 1. ] [0. 0.4 0.4],[0. 1. 1. ] [0.2 1.1 1.1] [0. 0.6 0.6],[ 0. 0.9 0.9] [-0.1 1. 1. ] [ 0. 0.6 0.6]
[0.1 0.9] [1. 0. ],[-0. 2.4 2.4] [-0.1 1.1 1.1] [ 0. 0.7 0.7],[0. 0.7 0.7] [0.2 1.1 1.2] [0. 0.7 0.7],[0. 1.4 1.4] [0. 0.9 0.9] [0. 0.4 0.4],[0. 0.8 0.8] [0.2 1.1 1.1] [0. 0.6 0.6],[-0. 2. 2. ] [-0.1 1. 1. ] [ 0. 0.6 0.6]
[0.8 0.2] [1. 0. ],[ 0. 0.8 0.8] [-0.1 1.1 1.1] [ 0.1 0.7 0.7],[0.1 1.8 1.8] [0.2 1.1 1.2] [0. 0.7 0.7],[0. 1.1 1.1] [0. 0.9 0.9] [0. 0.4 0.4],[0.1 1.7 1.7] [0.1 1.1 1.1] [0. 0.6 0.6],[ 0. 0.8 0.8] [-0.1 1. 1. ] [ 0. 0.5 0.5]


In [ ]:
for (i, π_b) in enumerate(πs):
    print("""$[\mask{{0.0}}{{{}}},\mask{{0.0}}{{{}}}]$ """.format(π_b[0,0], π_b[0,1]))
    for (j, π_e) in enumerate(πs):
        ix = i*len(πs)+j
        values = df_out_all_1[ix].iloc[[2,6,4], [2,3,4]].round(2).values
        print("""& \\scalebox{0.8}{$\\begin{matrix} """
              + """\mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}}""".format(*list(values.ravel())).replace('-', '\\shortminus ')
              + """ \end{matrix}$} """)
    print("""\\\\[12pt]""")

In [17]:
for (i, π_b) in enumerate(πs):
    print("""$[\mask{{0.0}}{{{}}},\mask{{0.0}}{{{}}}]$ """.format(π_b[0,0], π_b[0,1]))
    for (j, π_e) in enumerate(πs):
        ix = i*len(πs)+j
        values = df_out_all_1[ix].iloc[[2,6,4], [2,3,4]].round(1).values
        print("""& \\scalebox{0.8}{$\\begin{matrix} """
              + """\mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}}""".format(
                  *[int(x) if x.is_integer() else x for x in list(values.ravel())]).replace('-', '\\shortminus ')
              + """ \end{matrix}$} """)
    print("""\\\\[12pt]""")

$[\mask{0.0}{1.0},\mask{0.0}{0.0}]$ 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{0} & \mask{0.00}{0.7} & \mask{0.00}{0.7} \\ \mask{0.00}{\shortminus 0.1} & \mask{0.00}{1.1} & \mask{0.00}{1.1} \\ \mask{0.00}{0} & \mask{0.00}{0.7} & \mask{0.00}{0.7} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{\shortminus 0.5} & \mask{0.00}{0.4} & \mask{0.00}{0.6} \\ \mask{0.00}{0.2} & \mask{0.00}{1.1} & \mask{0.00}{1.1} \\ \mask{0.00}{0} & \mask{0.00}{0.7} & \mask{0.00}{0.7} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{\shortminus 0.2} & \mask{0.00}{0.5} & \mask{0.00}{0.5} \\ \mask{0.00}{0} & \mask{0.00}{0.9} & \mask{0.00}{0.9} \\ \mask{0.00}{0} & \mask{0.00}{0.4} & \mask{0.00}{0.4} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{\shortminus 0.4} & \mask{0.00}{0.4} & \mask{0.00}{0.6} \\ \mask{0.00}{0.2} & \mask{0.00}{1.1} & \mask{0.00}{1.1} \\ \mask{0.00}{0} & \mask{0.00}{0.6} & \mask{0.00}{0.6} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{\s

## Rs[2] setting

In [18]:
R, sigma = Rs[2], sigmas[2]

In [19]:
df_out_all_2 = []
for π_b, π_e in tqdm(list(itertools.product(πs, πs))):
    df_out = single_exp_setting(π_b, π_e)
    df_out_all_2.append(df_out)

  0%|          | 0/25 [00:00<?, ?it/s]<ipython-input-7-93125942f8f4>:115: RuntimeWarning: invalid value encountered in true_divide
  (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (N)
<ipython-input-7-93125942f8f4>:118: RuntimeWarning: invalid value encountered in true_divide
  (np.sum(w* π_e[x,a] / π_b_[x,a] * r) + np.nansum(wc* π_e[x,1-a] / π_b_[x,1-a] * rc)) / (np.sum(w* π_e[x,a] / π_b_[x,a]) + np.sum((wc* π_e[x,1-a] / π_b_[x,1-a])[c==1])),
  4%|▍         | 1/25 [00:07<02:52,  7.17s/it]<ipython-input-7-93125942f8f4>:33: RuntimeWarning: invalid value encountered in double_scalars
  WISs.append(np.sum(rho * r) / np.sum(rho))
100%|██████████| 25/25 [02:04<00:00,  4.98s/it]


In [20]:
df_tmp = pd.DataFrame(index=[str(π)[1:-1] for π in πs], columns=[str(π)[1:-1] for π in πs])
df_tmp.index.name = 'π_b'
df_tmp.columns.name = 'π_e'
for (i, π_b), (j, π_e) in itertools.product(enumerate(πs), enumerate(πs)):
    ix = i*len(πs)+j
    df_tmp.iloc[i,j] = str(df_out_all_2[ix].iloc[[2,6,4], [2,3,4]].round(1).values)[1:-1]
display(df_tmp.style.set_properties(**{'white-space': 'pre-wrap'}))

π_e,[1. 0.] [1. 0.],[0. 1.] [1. 0.],[0.5 0.5] [1. 0. ],[0.1 0.9] [1. 0. ],[0.8 0.2] [1. 0. ]
π_b,,,,,
[1. 0.] [1. 0.],[ 0. 0.7 0.7] [-0.1 1.1 1.1] [ 0. 0.7 0.7],[ 1. 0.4 1.1] [-0.3 2. 2. ] [ 0.1 1.1 1.1],[ 0.5 0.5 0.7] [-0.2 1. 1. ] [ 0.1 0.9 0.9],[ 0.9 0.4 1. ] [-0.3 1.7 1.8] [ 0.1 1.1 1.1],[ 0.2 0.6 0.7] [-0.2 0.9 0.9] [ 0. 0.8 0.8]
[0. 1.] [1. 0.],[ 0.5 0.4 0.6] [-0.1 1.1 1.1] [ 0. 0.7 0.7],[ 0.1 1.1 1.1] [-0.3 2. 2. ] [ 0.1 1.1 1.1],[ 0.3 0.7 0.7] [-0.2 1. 1. ] [ 0.1 0.9 0.9],[ 0.1 1. 1. ] [-0.3 1.7 1.8] [ 0.1 1.1 1.1],[ 0.4 0.4 0.6] [-0.2 0.9 0.9] [ 0. 0.8 0.8]
[0.5 0.5] [1. 0. ],[ 0. 1.1 1.1] [-0.1 1.1 1.1] [ 0. 0.7 0.7],[ 0.1 1.8 1.8] [-0.3 2. 2. ] [ 0.1 1.1 1.1],[ 0.1 1. 1. ] [-0.2 1. 1. ] [ 0.1 0.9 0.9],[ 0.1 1.6 1.6] [-0.3 1.7 1.8] [ 0.1 1.1 1.1],[ 0. 0.9 0.9] [-0.2 0.8 0.9] [ 0. 0.8 0.8]
[0.1 0.9] [1. 0. ],[-0. 2.4 2.4] [-0.1 1.1 1.1] [ 0. 0.7 0.7],[ 0.1 1.2 1.2] [-0.3 2. 2. ] [ 0.1 1.1 1.1],[ 0. 1.3 1.3] [-0.2 1. 1. ] [ 0.1 0.9 0.9],[ 0.1 1.1 1.1] [-0.3 1.7 1.8] [ 0.1 1.1 1.1],[ 0. 1.9 1.9] [-0.2 0.9 0.9] [ 0. 0.8 0.8]
[0.8 0.2] [1. 0. ],[ 0. 0.8 0.8] [-0.1 1.1 1.1] [ 0.1 0.7 0.7],[ 0.1 3.1 3.1] [-0.3 2. 2. ] [ 0. 1.1 1.1],[ 0. 1.5 1.5] [-0.2 1. 1. ] [ 0.1 0.9 0.9],[ 0.1 2.8 2.8] [-0.3 1.8 1.8] [ 0. 1.1 1.1],[ 0. 0.8 0.8] [-0.2 0.8 0.9] [ 0.1 0.8 0.8]


In [ ]:
for (i, π_b) in enumerate(πs):
    print("""$[\mask{{0.0}}{{{}}},\mask{{0.0}}{{{}}}]$ """.format(π_b[0,0], π_b[0,1]))
    for (j, π_e) in enumerate(πs):
        ix = i*len(πs)+j
        values = df_out_all_2[ix].iloc[[2,6,4], [2,3,4]].round(2).values
        print("""& \\scalebox{0.8}{$\\begin{matrix} """
              + """\mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}}""".format(*list(values.ravel())).replace('-', '\\shortminus ')
              + """ \end{matrix}$} """)
    print("""\\\\[12pt]""")

In [21]:
for (i, π_b) in enumerate(πs):
    print("""$[\mask{{0.0}}{{{}}},\mask{{0.0}}{{{}}}]$ """.format(π_b[0,0], π_b[0,1]))
    for (j, π_e) in enumerate(πs):
        ix = i*len(πs)+j
        values = df_out_all_2[ix].iloc[[2,6,4], [2,3,4]].round(1).values
        print("""& \\scalebox{0.8}{$\\begin{matrix} """
              + """\mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} \\\\ \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}} & \mask{{0.00}}{{{}}}""".format(
                  *[int(x) if x.is_integer() else x for x in list(values.ravel())]).replace('-', '\\shortminus ')
              + """ \end{matrix}$} """)
    print("""\\\\[12pt]""")

$[\mask{0.0}{1.0},\mask{0.0}{0.0}]$ 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{0} & \mask{0.00}{0.7} & \mask{0.00}{0.7} \\ \mask{0.00}{\shortminus 0.1} & \mask{0.00}{1.1} & \mask{0.00}{1.1} \\ \mask{0.00}{0} & \mask{0.00}{0.7} & \mask{0.00}{0.7} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{1} & \mask{0.00}{0.4} & \mask{0.00}{1.1} \\ \mask{0.00}{\shortminus 0.3} & \mask{0.00}{2} & \mask{0.00}{2} \\ \mask{0.00}{0.1} & \mask{0.00}{1.1} & \mask{0.00}{1.1} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{0.5} & \mask{0.00}{0.5} & \mask{0.00}{0.7} \\ \mask{0.00}{\shortminus 0.2} & \mask{0.00}{1} & \mask{0.00}{1} \\ \mask{0.00}{0.1} & \mask{0.00}{0.9} & \mask{0.00}{0.9} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{0.9} & \mask{0.00}{0.4} & \mask{0.00}{1} \\ \mask{0.00}{\shortminus 0.3} & \mask{0.00}{1.7} & \mask{0.00}{1.8} \\ \mask{0.00}{0.1} & \mask{0.00}{1.1} & \mask{0.00}{1.1} \end{matrix}$} 
& \scalebox{0.8}{$\begin{matrix} \mask{0.00}{0.2} &